In [1]:
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [2]:
# Load the Wine dataset
wine = datasets.load_wine()
X = wine.data
y = wine.target


In [3]:
# Adding the bias term for every instance
X_with_bias = np.c_[np.ones([len(X), 1]), X]

# Splitting the dataset manually
np.random.seed(42)
test_ratio = 0.2
validation_ratio = 0.2
total_size = len(X_with_bias)

test_size = int(total_size * test_ratio)
validation_size = int(total_size * validation_ratio)
train_size = total_size - test_size - validation_size

rnd_indices = np.random.permutation(total_size)

X_train = X_with_bias[rnd_indices[:train_size]]
y_train = y[rnd_indices[:train_size]]
X_valid = X_with_bias[rnd_indices[train_size:-test_size]]
y_valid = y[rnd_indices[train_size:-test_size]]
X_test = X_with_bias[rnd_indices[-test_size:]]
y_test = y[rnd_indices[-test_size:]]

In [4]:
# Converting class indices to one-hot vectors
def to_one_hot(y):
    n_classes = y.max() + 1
    m = len(y)
    Y_one_hot = np.zeros((m, n_classes))
    Y_one_hot[np.arange(m), y] = 1
    return Y_one_hot

Y_train_one_hot = to_one_hot(y_train)
Y_valid_one_hot = to_one_hot(y_valid)
Y_test_one_hot = to_one_hot(y_test)

In [5]:
# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train[:, 1:])
X_valid_scaled = scaler.transform(X_valid[:, 1:])
X_test_scaled = scaler.transform(X_test[:, 1:])

In [6]:
# Reassign the scaled data keeping the bias term unchanged
X_train[:, 1:] = X_train_scaled
X_valid[:, 1:] = X_valid_scaled
X_test[:, 1:] = X_test_scaled

In [7]:
# Softmax function
def softmax(logits):
    exps = np.exp(logits - np.max(logits, axis=1, keepdims=True))  # to prevent overflow
    exp_sums = exps.sum(axis=1, keepdims=True)
    return exps / exp_sums

In [8]:
# Training the model with Batch Gradient Descent
n_inputs = X_train.shape[1]
n_outputs = len(np.unique(y_train))
Theta = np.random.randn(n_inputs, n_outputs)

eta = 0.1
n_epochs = 5001
m = len(X_train)
epsilon = 1e-7
best_loss = np.inf

for epoch in range(n_epochs):
    logits = X_train.dot(Theta)
    Y_proba = softmax(logits)
    loss = -np.mean(np.sum(Y_train_one_hot * np.log(Y_proba + epsilon), axis=1))
    error = Y_proba - Y_train_one_hot
    gradients = 1/m * X_train.T.dot(error)
    Theta -= eta * gradients

    logits_valid = X_valid.dot(Theta)
    Y_proba_valid = softmax(logits_valid)
    loss_valid = -np.mean(np.sum(Y_valid_one_hot * np.log(Y_proba_valid + epsilon), axis=1))

    if epoch % 500 == 0:
        print(epoch, loss_valid)

    if loss_valid < best_loss:
        best_loss = loss_valid
    else:
        print("Early stopping!")
        break

0 3.862442613706157
500 0.10445253119329116
1000 0.08923322010615192
1500 0.08320593556823673
2000 0.08018307790883639
2500 0.07850098329889886
3000 0.07752243502780187
3500 0.07695289523072012
4000 0.07663838879619818
4500 0.07649105139117111
Early stopping!


In [9]:
# Evaluate the model on the test set
logits_test = X_test.dot(Theta)
Y_proba_test = softmax(logits_test)
y_predict = np.argmax(Y_proba_test, axis=1)

accuracy_score = np.mean(y_predict == y_test)
print("Test accuracy: ", accuracy_score)


Test accuracy:  0.9714285714285714
